In [11]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""
Example of using Estimator with multiple GPUs to distribute one model.
This example only runs if you have multiple GPUs to assign to.

"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection
import tensorflow as tf

X_FEATURE = 'x'  # Name of the input feature.


def my_model(features, labels, mode):
    """DNN with three hidden layers, and dropout of 0.1 probability.
  
    Note: If you want to run this example with multiple GPUs, Cuda Toolkit 7.0 and
    CUDNN 6.5 V2 from NVIDIA need to be installed beforehand.
  
    Args:
      features: Dict of input `Tensor`.
      labels: Label `Tensor`.
      mode: One of `ModeKeys`.
  
    Returns:
      `EstimatorSpec`.
    """
    # Create three fully connected layers respectively of size 10, 20, and 10 with
    # each layer having a dropout probability of 0.1.
    net = features[X_FEATURE]
    with tf.device('/device:GPU:1'):
        for units in [10, 20, 10]:
            net = tf.layers.dense(net, units=units, activation=tf.nn.relu)
            net = tf.layers.dropout(net, rate=0.1)

    with tf.device('/device:GPU:2'):
        # Compute logits (1 per class).
        logits = tf.layers.dense(net, 3, activation=None)

        # Compute predictions.
        predicted_classes = tf.argmax(logits, 1)
        if mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'class': predicted_classes,
                'prob': tf.nn.softmax(logits)
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

        # Convert the labels to a one-hot tensor of shape (length of features, 3)
        # and with a on-value of 1 for each one-hot vector of length 3.
        onehot_labels = tf.one_hot(labels, 3, 1, 0)
        # Compute loss.
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

        # Create training op.
        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
            train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

        # Compute evaluation metrics.
        eval_metric_ops = {
            'accuracy': tf.metrics.accuracy(
                labels=labels, predictions=predicted_classes)
        }
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [12]:

iris = datasets.load_iris()
x_train, x_test, y_train, y_test = model_selection.train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

classifier = tf.estimator.Estimator(model_fn=my_model)

# Train.
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={X_FEATURE: x_train}, y=y_train, num_epochs=None, shuffle=True)
classifier.train(input_fn=train_input_fn, steps=100)

# Predict.
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={X_FEATURE: x_test}, y=y_test, num_epochs=1, shuffle=False)
predictions = classifier.predict(input_fn=test_input_fn)
y_predicted = np.array(list(p['class'] for p in predictions))
y_predicted = y_predicted.reshape(np.array(y_test).shape)

# Score with sklearn.
score = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy (sklearn): {0:f}'.format(score))

# Score with tensorflow.
scores = classifier.evaluate(input_fn=test_input_fn)
print('Accuracy (tensorflow): {0:f}'.format(scores['accuracy']))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xa78c110>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpVIRRzp', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpVIRRzp/model.ckpt.
INFO:tensorflow:loss = 2.915372848510742, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpVIRRzp/model.ckpt.
INFO:tensorflow:Loss for final step: 0.1391492336988449.
INFO:tensorflow:Restoring parameters from /tmp/tmpVIRRzp/model.ckpt-100
Accuracy (sklearn): 0.966667
INFO:tensorflow:Starting ev

In [5]:
type(iris)

sklearn.datasets.base.Bunch

In [9]:
iris.data.shape

(150, 4)

In [7]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [10]:
iris.target.shape

(150,)